# COPT and LOLP Calculator

## import and settting printer

In [1]:
import itertools
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

In [2]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [3]:
# NOTE:
#   UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
#   COMMENT IF USING MANUAL INPUT
# csv input path
df = pd.read_csv('../data/COPT Case.csv')

# extract csv
capacities = df['Capacity (MW)'].tolist()
outage_rates = df['FOR (Outage)'].tolist()
status = [True] * len(df.index)

In [4]:
# # NOTE:
# #   UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
# #   COMMENT IF USING CSV INPUT
# # max capacity
# capacities = [
#     80,
#     70,
#     40,
#     50,
#     40,
#     50
# ]

# # forced outage rate
# outage_rates = [
#     0.1, 
#     0.2,
#     0.1,
#     0.2,
#     0.1,
#     0.1
# ]


# # status of generator,
# #     True: exist
# #     False: not exist
# status = [
#     True,
#     False,
#     True,
#     True,
#     True,
#     True,
# ]

In [5]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,50,0.100,True
2,50,0.100,True
3,50,0.100,True
4,30,0.100,True
5,10,0.070,True
6,20,0.070,True
7,10,0.100,True
8,5,0.100,True
9,10,0.070,True
10,10,0.070,True


In [6]:
# peak load or load at time considering
# used to calculate LOLP
demands = [590, 470, 585, 650, 660]

## Parameters

In [7]:
# NOTE:
#   There is two batch, bathc in evaluating of permutation and batch in making COPT.

BATCH_PRODUCT = 100  # batch size for each making table before combine duplicate
MIN_COVERAGE = 1 - 1e-12  # early stopper
BATCH_GENERATOR = 10

## COPT table

In [8]:
generator_list = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]
last_generator = 0

In [9]:
ls = generator_list[last_generator:last_generator+BATCH_GENERATOR]
last_generator += BATCH_GENERATOR
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

capacities_combination_generator = itertools.product(*capacities_fix)
probabilities_combinations_generator = itertools.product(*probabilities_fix)

table = []
with tqdm(total=MIN_COVERAGE) as pbar:
    sum_probability = 0
    while sum_probability <= MIN_COVERAGE:
        table += [[sum(i), np.prod(j)] for i, j in itertools.islice(zip(capacities_combination_generator,
                                                                        probabilities_combinations_generator),
                                                                    BATCH_PRODUCT)]

        # change table to array
        table = np.array(table)

        # sort table
        table = table[(-table[:, 0]).argsort(),:]

        # combine duplicate
        table = [[k,sum([x[1] for x in list(g)])] 
                for k,g in itertools.groupby(table, lambda x:x[0])]

        # progress bar
        sum_probability = np.sum(table, axis=0)[1]
        pbar.update(n=sum_probability - pbar.n)

# add all units off
if table[-1][0] != 0:
    table = np.vstack([table, [0, 1 - sum_probability]])
else:
    table = np.array(table)

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

In [10]:
columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,480.000,0.420,1.000,0.420
2,450.000,0.047,0.580,0.466
3,430.000,0.335,0.534,0.801
4,400.000,0.037,0.199,0.838
5,380.000,0.118,0.162,0.956
6,350.000,0.013,0.044,0.969
7,330.000,0.024,0.031,0.994
8,300.000,0.003,0.006,0.996
9,280.000,0.003,0.004,0.999
10,250.000,0.000,0.001,1.000


In [11]:
import copy

tables = []
tables.append(copy.copy(table)) 

In [12]:
ls = generator_list[last_generator:last_generator+BATCH_GENERATOR]
last_generator += BATCH_GENERATOR
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

capacities_combination_generator = itertools.product(*capacities_fix)
probabilities_combinations_generator = itertools.product(*probabilities_fix)

table = []
with tqdm(total=MIN_COVERAGE) as pbar:
    sum_probability = 0
    while sum_probability <= MIN_COVERAGE:
        table += [[sum(i), np.prod(j)] for i, j in itertools.islice(zip(capacities_combination_generator,
                                                                        probabilities_combinations_generator),
                                                                    BATCH_PRODUCT)]

        # change table to array
        table = np.array(table)

        # sort table
        table = table[(-table[:, 0]).argsort(),:]

        # combine duplicate
        table = [[k,sum([x[1] for x in list(g)])] 
                for k,g in itertools.groupby(table, lambda x:x[0])]

        # progress bar
        sum_probability = np.sum(table, axis=0)[1]
        pbar.update(n=sum_probability - pbar.n)

# add all units off
if table[-1][0] != 0:
    table = np.vstack([table, [0, 1 - sum_probability]])
else:
    table = np.array(table)

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

In [13]:
columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,190.000,0.448,1.000,0.448
2,180.000,0.050,0.552,0.498
3,170.000,0.326,0.502,0.823
4,160.000,0.036,0.177,0.860
5,150.000,0.104,0.140,0.964
6,140.000,0.012,0.036,0.976
7,130.000,0.019,0.024,0.995
8,120.000,0.002,0.005,0.997
9,110.000,0.002,0.003,1.000
10,100.000,0.000,0.000,1.000


In [15]:
tables.append(copy.copy(table)) 

In [18]:
# faster than flatten + transpose
table = np.hstack(((tables[0][:, 0, None] + tables[1][:, 0]).reshape(-1,1),
                   (tables[0][:, 1, None] * tables[1][:, 1]).reshape(-1,1)))

# sort table
table = table[(-table[:, 0]).argsort(),:]

# combine duplicate
table = np.array([[k,sum([x[1] for x in list(g)])] 
                   for k,g in itertools.groupby(table, lambda x:x[0])])

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1)).to_csv('../results/merged.csv')

In [26]:
# NOTE:
#   There is two batch, bathc in evaluating of permutation and batch in making COPT.

BATCH_PRODUCT = 100  # batch size for each making table before combine duplicate
MIN_COVERAGE = 1 - 1e-12  # early stopper
BATCH_GENERATOR = 20

In [27]:
generator_list = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]
last_generator = 0

In [28]:
ls = generator_list[last_generator:last_generator+BATCH_GENERATOR]
last_generator += BATCH_GENERATOR
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

capacities_combination_generator = itertools.product(*capacities_fix)
probabilities_combinations_generator = itertools.product(*probabilities_fix)

table = []
with tqdm(total=MIN_COVERAGE) as pbar:
    sum_probability = 0
    while sum_probability <= MIN_COVERAGE:
        table += [[sum(i), np.prod(j)] for i, j in itertools.islice(zip(capacities_combination_generator,
                                                                        probabilities_combinations_generator),
                                                                    BATCH_PRODUCT)]

        # change table to array
        table = np.array(table)

        # sort table
        table = table[(-table[:, 0]).argsort(),:]

        # combine duplicate
        table = [[k,sum([x[1] for x in list(g)])] 
                for k,g in itertools.groupby(table, lambda x:x[0])]

        # progress bar
        sum_probability = np.sum(table, axis=0)[1]
        pbar.update(n=sum_probability - pbar.n)

# add all units off
if table[-1][0] != 0:
    table = np.vstack([table, [0, 1 - sum_probability]])
else:
    table = np.array(table)

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1)).to_csv('../results/single20units.csv')

  0%|          | 0/0.999999999999 [00:00<?, ?it/s]

In [29]:
pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,670.000,0.188,1.000,0.188
2,660.000,0.021,0.812,0.209
3,650.000,0.137,0.791,0.346
4,640.000,0.036,0.654,0.382
5,630.000,0.046,0.618,0.428
...,...,...,...,...
63,50.000,0.000,0.000,1.000
64,40.000,0.000,0.000,1.000
65,30.000,0.000,0.000,1.000
66,20.000,0.000,0.000,1.000


## LOLP

In [ ]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [ ]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
lolp

[1.0000000000000002,
 0.09932706030330078,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002]

In [ ]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 4.0993
